<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [3]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [4]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [5]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [6]:
from scipy.sparse import csr_matrix

In [7]:
csr_matrix?

In [8]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [9]:
full_sites_sparse.shape

(336358, 48371)

In [10]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [11]:
X_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

In [12]:
X_test_sparse.shape

(82797, 48371)

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [13]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse, y_train)

Wall time: 4.86 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [14]:
# predict class number
logit.predict(X_test_sparse[:15, :])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [15]:
# predict class probability
logit.predict_proba(X_test_sparse[:15, :])[:, 1]

array([2.21956510e-03, 2.51893030e-09, 6.16012354e-09, 1.32267446e-08,
       2.72907620e-05, 1.51179473e-04, 4.42376463e-04, 1.01245810e-04,
       7.77306796e-04, 1.04987720e-01, 2.37985641e-05, 9.55262104e-05,
       2.86682554e-04, 3.58981621e-01, 3.47103894e-05])

In [16]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C=C, n_jobs=-1, random_state=seed)
    logit.fit(X_train, y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [17]:
%%time
get_auc_lr_valid(X_train_sparse, y_train)

Wall time: 2.98 s


0.9197948027277921

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [18]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label='session_id'):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [19]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse, y_train)

Wall time: 3.27 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [20]:
test_pred = logit.predict_proba(X_test_sparse)[:, 1]

In [21]:
test_pred

array([2.21956510e-03, 2.51893030e-09, 6.16012354e-09, ...,
       8.43164771e-03, 3.87747333e-04, 1.29532177e-05])

In [22]:
# write submission file with another construction
pd.Series(test_pred, index=range(1, test_pred.shape[0]+1), 
          name='target').to_csv('benchmark1.csv', header=True, index_label='session_id')

In [23]:
!head benchmark1.csv

session_id,target
1,0.0022195650980852237
2,2.51893030097545e-09
3,6.160123540539234e-09
4,1.3226744611418958e-08
5,2.7290762036701066e-05
6,0.0001511794731709091
7,0.00044237646286624915
8,0.00010124581007160785
9,0.0007773067962455178


Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [24]:
new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

In [25]:
train_df['time1'].head()

session_id
21669    2013-01-12 08:05:57
54843    2013-01-12 08:37:23
77292    2013-01-12 08:50:13
114021   2013-01-12 08:50:17
146670   2013-01-12 08:50:20
Name: time1, dtype: datetime64[ns]

In [26]:
train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month).head()

session_id
21669     201301
54843     201301
77292     201301
114021    201301
146670    201301
Name: time1, dtype: int64

In [27]:
new_feat_train['year_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)

In [28]:
new_feat_train.head()

,year_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [29]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1, 1))

new_feat_train['year_month_scaled'] = scaler.transform(
    new_feat_train['year_month'].values.reshape(-1, 1))
new_feat_test['year_month_scaled'] = scaler.transform(
    new_feat_test['year_month'].values.reshape(-1, 1))

In [30]:
new_feat_train.head()

,year_month,year_month_scaled
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


In [31]:
X_train_sparse.shape, new_feat_train['year_month_scaled'].values.reshape(-1, 1).shape

((253561, 48371), (253561, 1))

In [32]:
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, 
                             new_feat_train['year_month_scaled'].values.reshape(-1, 1)]))

In [33]:
X_train_sparse.shape, X_train_sparse_new.shape

((253561, 48371), (253561, 48372))

In [34]:
%%time
get_auc_lr_valid(X_train_sparse_new, y_train)

Wall time: 3.62 s


0.9198902054055882

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [35]:
train_df['time1'].head(), train_df['time1'].apply(lambda sm: sm.hour).tail()

(session_id
 21669    2013-01-12 08:05:57
 54843    2013-01-12 08:37:23
 77292    2013-01-12 08:50:13
 114021   2013-01-12 08:50:17
 146670   2013-01-12 08:50:20
 Name: time1, dtype: datetime64[ns],
 session_id
 12224     23
 164438    23
 12221     23
 156968    23
 204762    23
 Name: time1, dtype: int64)

In [36]:
new_feat_train['start_hour'] = train_df['time1'].apply(lambda sm: sm.hour)
new_feat_test['start_hour'] = test_df['time1'].apply(lambda sm: sm.hour)

In [37]:
new_feat_train.head(), new_feat_train.tail()

(            year_month  year_month_scaled  start_hour
 session_id                                           
 21669           201301          -1.744405           8
 54843           201301          -1.744405           8
 77292           201301          -1.744405           8
 114021          201301          -1.744405           8
 146670          201301          -1.744405           8,
             year_month  year_month_scaled  start_hour
 session_id                                           
 12224           201404           0.681626          23
 164438          201404           0.681626          23
 12221           201404           0.681626          23
 156968          201404           0.681626          23
 204762          201404           0.681626          23)

In [38]:
scaler.fit(new_feat_train['start_hour'].values.reshape(-1, 1))

new_feat_train['start_hour_scaled'] = scaler.transform(
    new_feat_train['start_hour'].values.reshape(-1, 1))
new_feat_test['start_hour_scaled'] = scaler.transform(
    new_feat_test['start_hour'].values.reshape(-1, 1))

In [39]:
new_feat_train['morning'] = new_feat_train['start_hour'].apply(lambda m: 1 if m < 12 else 0)
new_feat_test['morning'] = new_feat_test['start_hour'].apply(lambda m: 1 if m < 12 else 0)

In [68]:
scaler.fit(new_feat_train['morning'].values.reshape(-1, 1))

new_feat_train['morning_scaled'] = scaler.transform(
    new_feat_train['morning'].values.reshape(-1,1))
new_feat_test['morning_scaled'] = scaler.transform(
    new_feat_test['morning'].values.reshape(-1,1))

In [94]:
new_feat_train.head()

,year_month,year_month_scaled,start_hour,start_hour_scaled,morning,morning_scaled
session_id,,,,,,
21669,201301,-1.744405,8,-1.357366,1,1.039061
54843,201301,-1.744405,8,-1.357366,1,1.039061
77292,201301,-1.744405,8,-1.357366,1,1.039061
114021,201301,-1.744405,8,-1.357366,1,1.039061
146670,201301,-1.744405,8,-1.357366,1,1.039061


In [70]:
X_train_sparse.shape, new_feat_train.shape

((253561, 48371), (253561, 6))

In [42]:
X_train_sparse_new_2 = csr_matrix(hstack([X_train_sparse, 
                                          new_feat_train[['year_month_scaled', 'start_hour_scaled']]]))

In [43]:
X_train_sparse.shape, X_train_sparse_new_2.shape

((253561, 48371), (253561, 48373))

In [44]:
%%time
get_auc_lr_valid(X_train_sparse_new_2, y_train)

Wall time: 3.36 s


0.9573322845076919

In [71]:
X_train_sparse_new_3 = csr_matrix(hstack([X_train_sparse, 
                                         new_feat_train[['year_month_scaled', 'morning_scaled']]]))

In [72]:
X_train_sparse.shape, X_train_sparse_new_3.shape

((253561, 48371), (253561, 48373))

In [73]:
%%time
get_auc_lr_valid(X_train_sparse_new_3, y_train)

Wall time: 4.65 s


0.9483688104493708

In [102]:
X_train_sparse_new_4 = csr_matrix(hstack([X_train_sparse,
                                         new_feat_train[['year_month_scaled', 
                                                         'start_hour_scaled', 
                                                         'morning_scaled']]]))
X_test_sparse_new_4 = csr_matrix(hstack([X_test_sparse,
                                         new_feat_test[['year_month_scaled', 
                                                         'start_hour_scaled', 
                                                         'morning_scaled']]]))

In [103]:
X_train_sparse.shape, X_train_sparse_new_4.shape

((253561, 48371), (253561, 48374))

In [104]:
X_test_sparse.shape, X_test_sparse_new_4.shape

((82797, 48371), (82797, 48374))

In [77]:
%%time
get_auc_lr_valid(X_train_sparse_new_4, y_train)
#AUC with nonscaled morning: 0.9585507820000507

Wall time: 3.5 s


0.9586546380796769

In [52]:
%%time
logit.fit(X_train_sparse_new_4, y_train)

Wall time: 3.71 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [53]:
test_pred_2 = logit.predict_proba(X_test_sparse_new_4)[:, 1]

In [54]:
test_pred_2

array([5.26264675e-05, 6.26559754e-08, 2.43439689e-09, ...,
       2.04955017e-04, 6.29702889e-06, 2.71673197e-07])

In [55]:
write_to_submission_file(test_pred_2, 'benchmark2.csv',
                             target='target', index_label="session_id")

In [56]:
!head benchmark2.csv

session_id,target
1,5.2626467522897445e-05
2,6.265597542283873e-08
3,2.4343968919068132e-09
4,2.297726836482974e-10
5,2.297929005433674e-05
6,3.5575931144026895e-06
7,0.0004852714570709584
8,0.00014757049918163388
9,1.4825073195417786e-05


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [78]:
get_auc_lr_valid(X_train_sparse_new_4, y_train, C=1.0)
#AUC with non scaled 'morning': 0.9585507820000507

0.9586546380796769

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [79]:
#AUC, C with non scaled 'morning': (0.9609942679897447, 0.1261856883066021)
#AUC, C with scaled 'morning':     (0.9607916882530321, 0.1261856883066021)
C_max = 1.0
AUC_max = 0.0
for C in np.logspace(-3, 1, 100):
    AUC = get_auc_lr_valid(X_train_sparse_new_4, y_train, C=C)
    if AUC_max < AUC: 
        AUC_max = AUC
        C_max = C

AUC_max, C_max

(0.9607916882530321, 0.1261856883066021)

In [85]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
c_values = np.logspace(-3, 1, 10)

logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, max_iter=1000, verbose=1, n_jobs=-1)
logit_searcher.fit(X_train_sparse_new_4, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.1min finished


LogisticRegressionCV(Cs=array([1.00000000e-03, 2.78255940e-03, 7.74263683e-03, 2.15443469e-02,
       5.99484250e-02, 1.66810054e-01, 4.64158883e-01, 1.29154967e+00,
       3.59381366e+00, 1.00000000e+01]),
                     class_weight=None,
                     cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=1000, multi_class='auto',
                     n_jobs=-1, penalty='l2', random_state=None, refit=True,
                     scoring=None, solver='lbfgs', tol=0.0001, verbose=1)

In [88]:
logit_searcher.C_, logit_searcher.scores_

(array([3.59381366]),
 {1: array([[0.99092935, 0.99090963, 0.99087019, 0.99112654, 0.9914026 ,
          0.99175754, 0.99197444, 0.99221107, 0.99232938, 0.99223079],
         [0.99094889, 0.99094889, 0.99092917, 0.99118552, 0.99152074,
          0.9916785 , 0.9920926 , 0.99230951, 0.99252642, 0.99238839],
         [0.99094889, 0.99094889, 0.99098833, 0.99132355, 0.99154046,
          0.9916785 , 0.99199401, 0.99225035, 0.99236867, 0.99223064],
         [0.99094889, 0.99092917, 0.99094889, 0.99124468, 0.99144187,
          0.99159962, 0.99189541, 0.99201372, 0.99189541, 0.9916785 ],
         [0.99092917, 0.99090945, 0.99085029, 0.9911658 , 0.99144187,
          0.9916785 , 0.99203344, 0.99236867, 0.99252642, 0.99248699]])})

Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [105]:
# AUC with scaled 'morning': 0.9607916882530321, gives higher result on leaderboard
# AUC with non scaled 'morning': 0.9609942679897447
get_auc_lr_valid(X_train_sparse_new_4, y_train, C=0.1261856883066021)

0.9607916882530321

In [99]:
#A3 baseline 2: 0.92784
logit_2 = LogisticRegression(C=0.1261856883066021, random_state=17, n_jobs=-1)
logit_2.fit(X_train_sparse_new_4, y_train)
test_pred_2 = logit_2.predict_proba(X_test_sparse_new_4)[:, 1]
test_pred_2

array([3.05913809e-04, 3.52363789e-06, 1.77867880e-06, ...,
       6.41287437e-04, 7.86418462e-05, 1.10998940e-05])

In [100]:
#benchmark2-2.csv (logit optimized + 3 feature (included scaled 'morning') )public score 0.92700
#benchmark2-3.csv (logit optimized + 3 feature (included non scaled 'morning') )public score 0.92595
write_to_submission_file(test_pred_2, 'benchmark2-3.csv') #Public srore 0.92595

In [1]:
!head benchmark2-3.csv

session_id,target
1,0.0003059138085530719
2,3.523637885183798e-06
3,1.778678795506902e-06
4,1.0405829640001881e-06
5,0.0002353719762725874
6,6.062759732714576e-05
7,0.0022566164641240364
8,0.0018459467211361961
9,0.00012147391491643461
